In [ ]:
## Anomaly detection
#### Loading the csv and libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
ctr = pd.read_csv(r'C:\Users\Rick-Royal\Documents\PROJECT\ctr_eda1.csv')
ctr.head()
ctr.columns
# from ctr, check for row entries containing punj in the organizationname column
ctr.loc[ctr['OrganizationName'].str.contains('punj', case=False)]
ctr.info()
ctr.nunique()
ctr.columns
# include the figure size
plt.figure(figsize=(10,10))
# plot the correlation matrix
corr = ctr.corr()
sns.heatmap(corr,
        xticklabels=corr.columns,
        yticklabels=corr.columns)


corr
# create a new dataframe without unique identifiers
ctr = ctr.drop(['ReportDate', 'ReportNo', '_Transaction_day_of_week',
       'ItemNo', 'FullName', 'AccountName', 'IDTypeAndNo','currency', 'TransactionDate',
       'TransDescription', 'Report_year'], axis=1)
# from ctr, label encode the following columns ,
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
ctr['Report_day_of_week'] = le.fit_transform(ctr['Report_day_of_week'])
# ctr['_Transaction_day_of_week'] = le.fit_transform(ctr['_Transaction_day_of_week'])
ctr['Industry'] = le.fit_transform(ctr['Industry'])
ctr['OrganizationName'] = le.fit_transform(ctr['OrganizationName'])
# One-hot encode the specified columns
ctr_encoded = pd.get_dummies(ctr, columns=[ 'CurrencyType', 'person_type', 'Account_type']) # 

# Check the resulting dataframe
print(ctr_encoded.head())
ctr_encoded.columns
ctr_encoded.dtypes
# import pandas as pd
# from sklearn.preprocessing import StandardScaler, MinMaxScaler

# # Load your dataframe (replace this with your actual dataframe)
# # ctr = pd.read_csv("your_data.csv")

# # Create instances of StandardScaler and MinMaxScaler
# standard_scaler = StandardScaler()
# minmax_scaler = MinMaxScaler()

# # Select the columns you want to scale (numerical columns)
# columns_to_scale = ['col1', 'col2', 'col3']  # Replace with your actual column names

# # Apply StandardScaler
# ctr_standard_scaled = ctr.copy()
# ctr_standard_scaled[columns_to_scale] = standard_scaler.fit_transform(ctr[columns_to_scale])

# # Apply MinMaxScaler
# ctr_minmax_scaled = ctr.copy()
# ctr_minmax_scaled[columns_to_scale] = minmax_scaler.fit_transform(ctr[columns_to_scale])

# # Check the resulting dataframes
# print("Standard Scaled Data:")
# print(ctr_standard_scaled.head())

# print("MinMax Scaled Data:")
# print(ctr_minmax_scaled.head())

import pandas as pd
from sklearn.preprocessing import StandardScaler

# Create instances of StandardScaler and MinMaxScaler
standard_scaler = StandardScaler()

# Select the columns you want to scale (numerical columns)
columns_to_scale = ['Amount', 'days_diff', 'Transaction_year', 'Transaction_month',
       'Transaction_day', 'Transaction_day_of_week',
       'Report_month', 'Report_day', 'Report_day_of_week', 'Amount_KES']  # Replace with your actual column names

# Apply StandardScaler
ctr_standard_scaled = ctr_encoded.copy()
ctr_standard_scaled[columns_to_scale] = standard_scaler.fit_transform(ctr_encoded[columns_to_scale])

# Check the resulting dataframes
print("Standard Scaled Data:")
print(ctr_standard_scaled.head())



ctr_standard_scaled.head()
# from ctr_standard_scaled, split the data into train and test for an unlabeled dataset
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(ctr_standard_scaled, test_size=0.3, random_state=42)

### Anomaly detection
# Import libraries for isolation forest
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Import silhouette_score, calinski_harabasz_score, davies_bouldin_score, kolmogorov_smirnov_test
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

# Create an isolation forest model
isolation_forest = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)

# Fit the model
isolation_forest.fit(X_train)

# Predict the model
y_pred = isolation_forest.predict(X_test)


# Importing Libraries
from pyod.models.iforest import IForest
from pyod.utils.data import generate_data

from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize

anomaly_proportion = 0.001

# Training IForest indicator
clf_name = 'Anomaly detection - Isolation forest'
clf = IForest(contamination = anomaly_proportion)

# Fit the model
clf.fit(X_train)


X_train['y_pred'] = clf.labels_ #binary labels (0=inlier, 1=outlier)
X_train['y_scores'] = clf.decision_scores_ # raw outlier scores. The bigger the number the greater the anomaly.
xx , yy = np.meshgrid(np.linspace(0, 11, 200), np.linspace(0, 180000, 200))

# decision function calculates the raw anomaly score for every point
Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])*-1
Z = Z.reshape(xx.shape)


threshold = (X_train.loc[X_train['y_pred'] == 1, 'y_scores'].min()*-1)/2 + (X_train.loc[X_train['y_pred'] == 0, 'y_scores'].max()*-1)/2


subplot = plt.subplot(1, 1, 1)

# fill blue colormap from minimum anomaly score to threshold value
subplot.contourf(xx, yy, Z, levels = np.linspace(Z.min(), threshold, 10),cmap=plt.cm.Blues_r)

# draw red contour line where anomaly score is equal to threshold
a = subplot.contour(xx, yy, Z, levels=[threshold],linewidths=2, colors='red')

# fill orange contour lines where range of anomaly score is from threshold to maximum anomaly score
subplot.contourf(xx, yy, Z, levels=[threshold, Z.max()],colors='orange')

msk = X_train['y_pred'] == 0
x = X_train.loc[msk, ['Amount', 'days_diff', 'Transaction_year', 'Transaction_month',
       'Transaction_day', 'Transaction_day_of_week', 'Report_year',
       'Report_month', 'Report_day', 'Report_day_of_week', 'Amount_KES',
       'person_type_entity', 'person_type_human', 'Account_type_business',
       'Account_type_personal', 'currency_AUD', 'currency_CAD', 'currency_CHF',
       'currency_DKK', 'currency_EGP', 'currency_EUR', 'currency_GBP',
       'currency_HKK', 'currency_INR', 'currency_JPY', 'currency_KES',
       'currency_KOW', 'currency_NPR', 'currency_PAR', 'currency_PKR',
       'currency_QTR', 'currency_RUB', 'currency_SDL', 'currency_SEK',
       'currency_SFR', 'currency_UAE', 'currency_USD', 'currency_ZAR']].values

# scatter plot of inliers with white dots
b = subplot.scatter(x[:, 0], x[:, 1], c='white',s=20, edgecolor='k') 


msk = X_train['y_pred'] == 1
x = X_train.loc[msk, ['Amount', 'days_diff', 'Transaction_year', 'Transaction_month',
       'Transaction_day', 'Transaction_day_of_week', 'Report_year',
       'Report_month', 'Report_day', 'Report_day_of_week', 'Amount_KES',
       'person_type_entity', 'person_type_human', 'Account_type_business',
       'Account_type_personal', 'currency_AUD', 'currency_CAD', 'currency_CHF',
       'currency_DKK', 'currency_EGP', 'currency_EUR', 'currency_GBP',
       'currency_HKK', 'currency_INR', 'currency_JPY', 'currency_KES',
       'currency_KOW', 'currency_NPR', 'currency_PAR', 'currency_PKR',
       'currency_QTR', 'currency_RUB', 'currency_SDL', 'currency_SEK',
       'currency_SFR', 'currency_UAE', 'currency_USD', 'currency_ZAR']].values

# scatter plot of outliers with black dots
c = subplot.scatter(x[:, 0], x[:, 1], c='black',s=20, edgecolor='r')
subplot.axis('tight')

subplot.legend(
    [a.collections[0], b, c],
    ['learned decision function', 'inliers', 'outliers'],
    prop=plt.font_manager.FontProperties(size=10),
    loc='upper right')
subplot.set_title(clf_name)
subplot.set_xlim((0, 11))
subplot.set_ylim((0, 180000))

subplot.set_xlabel("5-day count of withdrawal transactions.")
subplot.set_ylabel("5-day sum of withdrawal transactions")
# from ctr, check for row entries containing punj in the organizationname column
ctr.loc[ctr['OrganizationName'].str.contains('punj', case=False)]

### PCA Decomposition
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Assuming ctr_encoded is a pandas DataFrame
# Standardize the features for better PCA performance
scaler = StandardScaler()
ctr_standard_scaled = scaler.fit_transform(ctr_encoded)

# Initialize PCA and fit it to the standardized data
pca = PCA()
pca.fit(ctr_standard_scaled)

# Calculate the cumulative explained variance
cumulative_explained_variance = np.cumsum(pca.explained_variance_ratio_)

# Find the efficient number of components to explain more than 99% of the variance
n_components = np.where(cumulative_explained_variance >= 0.99)[0][0] + 1

# Perform PCA with the efficient number of components
pca = PCA(n_components=n_components)
ctr_pca = pca.fit_transform(ctr_standard_scaled)

# Use k-means clustering on the reduced dataset
kmeans = KMeans(n_clusters=3)  # Choose the desired number of clusters (k)
kmeans.fit(ctr_pca)

# Get the cluster assignments for each data point
clusters = kmeans.labels_

import matplotlib.pyplot as plt

# Assuming ctr_pca is a 2D numpy array with the PCA-transformed data
# and clusters is a 1D numpy array with the cluster assignments

# Create a scatter plot of the first two principal components
plt.figure(figsize=(10, 7))
scatter = plt.scatter(ctr_pca[:, 0], ctr_pca[:, 1], c=clusters, cmap='viridis', alpha=0.5)

# Add a colorbar with cluster labels
colorbar = plt.colorbar(scatter)
colorbar.set_ticks([0, 1, 2])  # Change this to the number of clusters - 1
colorbar.set_ticklabels(['Cluster 0', 'Cluster 1', 'Cluster 2'])  # Change this to the number of clusters

# Label the axes with the principal components
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

# Set the title and display the plot
plt.title('K-means Clustering with PCA-Reduced Data')
plt.show()

# 1. Cluster 1- Medium risk,
# 2. Cluster 2- low risk,
# 3. Cluster 0- high risk,
plt.figure(figsize=(10, 7))

if ctr_pca.shape[1] > 1:
    scatter = plt.scatter(ctr_pca[:, 0], ctr_pca[:, 1], c=clusters, cmap='viridis', alpha=0.5)
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
else:
    scatter = plt.scatter(ctr_pca[:, 0], np.zeros_like(ctr_pca[:, 0]), c=clusters, cmap='viridis', alpha=0.5)
    plt.xlabel('Principal Component 1')
    plt.yticks([])

plt.title('PCA Scatter Plot')
plt.colorbar(scatter, label='Cluster', ticks=range(kmeans.n_clusters))

plt.show()

# Assuming ctr is the original dataset as a pandas DataFrame
# and clusters is a 1D numpy array with the cluster assignments

# Add the cluster assignments as a new column to the original dataset
ctr['cluster'] = clusters

# Print the merged dataset with cluster assignments
print(ctr.head())

# plot a scatter plot of the 2 features and color the points by cluster
plt.figure(figsize=(10, 7))
scatter = plt.scatter(ctr['days_diff'], ctr['Amount_KES'], c=ctr['cluster'], cmap='viridis', alpha=0.5)

# Add a colorbar with cluster labels
colorbar = plt.colorbar(scatter)
colorbar.set_ticks([0, 1, 2])  # Change this to the number of clusters - 1
colorbar.set_ticklabels(['Cluster 0', 'Cluster 1', 'Cluster 2'])  # Change this to the number of clusters

# Label the axes    
plt.xlabel('days_diff')
plt.ylabel('Amount_KES')

# Set the title and display the plot
plt.title('K-means Clustering with PCA-Reduced Data')
plt.show()


# plot a bar plot of the cluster labels and AMount_KES
plt.figure(figsize=(10, 7))
sns.barplot(x=ctr['cluster'], y=ctr['Amount_KES'])

# Label the axes
plt.xlabel('Cluster')
plt.ylabel('Amount_KES')

# Set the title and display the plot    
plt.title('Average Amount_KES per Cluster')
plt.show()

ctr_1 = pd.read_csv(r'C:\Users\Rick-Royal\Documents\PROJECT\ctr_eda1.csv')

# Add the cluster assignments as a new column to the original dataset
ctr_1['cluster'] = clusters

# Print the merged dataset with cluster assignments
print(ctr_1.columns)
# ctr_1.to_csv('ctr_cluster_t2.csv', index=False)
# # from ctr_encoded, conduct PCA by finding the efficient number of components
# # select the number of components such that the amount of variance that needs to be explained is greater than 95%
# # the PCA will be used to reduce the number of features in the dataset
# # the PCA dataset will be used in finding clusters in the dataset using k-means clustering

# from sklearn.decomposition import PCA

# pca = PCA(n_components=0.95)
# pca.fit(X_train)

# # the number of components that explain 95% of the variance in the dataset
# pca.n_components_

# # the variance explained by each of the components
# pca.explained_variance_ratio_

# # the cumulative variance explained by the components
# pca.explained_variance_ratio_.cumsum()

# # the components
# pca.components_

# # the transformed dataset
# X_train_pca = pca.transform(X_train)

# # the transformed dataset
# X_test_pca = pca.transform(X_test)


#### EDA
# from ctr_1, plot a scatter plot of amount_kes and days_diff and color the points by cluster
plt.figure(figsize=(10, 7))
scatter = plt.scatter(ctr_1['days_diff'], ctr_1['Amount_KES'], c=ctr_1['cluster'], cmap='viridis', alpha=0.5)

# Add a colorbar with cluster labels
colorbar = plt.colorbar(scatter)
colorbar.set_ticks([0, 1, 2])  # Change this to the number of clusters - 1
colorbar.set_ticklabels(['Cluster 0', 'Cluster 1', 'Cluster 2'])  # Change this to the number of clusters

# Label the axes
plt.xlabel('days_diff')
plt.ylabel('Amount_KES')

# Set the title and display the plot
plt.title('Amount_KES vs days_diff')
plt.show()

import seaborn as sns

# Set the plot style and size
sns.set(style='whitegrid')
plt.figure(figsize=(10, 7))

# Create a bar plot of the number of transactions per cluster
ax = sns.countplot(x='cluster', data=ctr)

# Set the x and y-axis labels and plot title
ax.set_xlabel('Cluster')
ax.set_ylabel('Number of Transactions')
ax.set_title('Number of Transactions per Cluster')

# Display the plot
plt.show()

import seaborn as sns

# Set the plot style and size
sns.set(style='whitegrid')
plt.figure(figsize=(12, 7))

# Create a histogram of transaction amounts for each cluster
ax = sns.histplot(data=ctr, x='Amount_KES', hue='cluster', bins=50, kde=True, alpha=0.6, multiple='stack')

# Set the x and y-axis labels and plot title
ax.set_xlabel('Transaction Amount (KES)')
ax.set_ylabel('Count')
ax.set_title('Distribution of Transaction Amounts by Cluster')

# Display the plot
plt.show()

import seaborn as sns

# Set the plot style and size
sns.set(style='whitegrid')
plt.figure(figsize=(12, 7))

# Create a box plot of transaction amounts for each cluster
ax = sns.boxplot(x='cluster', y='Amount_KES', data=ctr)

# Set the x and y-axis labels and plot title
ax.set_xlabel('Cluster')
ax.set_ylabel('Transaction Amount (KES)')
ax.set_title('Distribution of Transaction Amounts by Cluster')

# Display the plot
plt.show()

# Create a new figure for the days_diff box plot
plt.figure(figsize=(12, 7))

# Create a box plot of days_diff for each cluster
ax = sns.boxplot(x='cluster', y='days_diff', data=ctr)

# Set the x and y-axis labels and plot title
ax.set_xlabel('Cluster')
ax.set_ylabel('Days Difference')
ax.set_title('Distribution of Days Difference by Cluster')

# Display the plot
plt.show()

import seaborn as sns

# Calculate the average transaction amount for each industry and cluster combination
average_amounts = ctr_1.groupby(['Industry', 'cluster'])['Amount_KES'].mean().unstack()

# Set the plot style and size
sns.set(style='whitegrid')
plt.figure(figsize=(12, 7))

# Create a heatmap of the average transaction amounts
ax = sns.heatmap(average_amounts, annot=True, fmt='.2f', cmap='YlGnBu', linewidths=0.5)

# Set the x and y-axis labels and plot title
ax.set_xlabel('Cluster')
ax.set_ylabel('Industry')
ax.set_title('Average Transaction Amount by Industry and Cluster')

# Display the plot
plt.show()

# In this code, we first calculate the average transaction amount (Amount_KES) for each industry and cluster combination using groupby() and mean() functions. Then, we create a heatmap using sns.heatmap. The annot parameter is set to True, which displays the average transaction amount in each cell. The fmt parameter is set to '.2f', which formats the numbers to two decimal places. The cmap parameter is set to 'YlGnBu', which controls the colormap used for the heatmap. The linewidths parameter is set to 0.5, which adds a thin border between cells.

# Adjust the parameters as needed to customize the appearance of the heatmap.
from sklearn.metrics import silhouette_score

# Calculate the silhouette score
sil_score = silhouette_score(ctr_pca, clusters)

print("Silhouette score:", sil_score)

ctr_1.columns
ctr_1.info()
import seaborn as sns

# Convert TransactionDate to datetime and extract the month and year
ctr_1['TransactionDate'] = pd.to_datetime(ctr_1['TransactionDate'])
ctr_1['Transaction_month_year'] = ctr_1['TransactionDate'].dt.to_period('M').astype(str)

# Calculate the total transaction amount and the number of transactions per month for each cluster
monthly_amounts = ctr_1.groupby(['Transaction_month_year', 'cluster'])['Amount_KES'].sum().reset_index()
monthly_transactions = ctr_1.groupby(['Transaction_month_year', 'cluster']).size().reset_index(name='transaction_count')

# Set the plot style and size
sns.set(style='whitegrid')
plt.figure(figsize=(14, 7))

# Create a line plot of the total transaction amount per month for each cluster
ax = sns.lineplot(x='Transaction_month_year', y='Amount_KES', hue='cluster', data=monthly_amounts)

# Set the x and y-axis labels and plot title
ax.set_xlabel('Month/Year')
ax.set_ylabel('Total Transaction Amount (KES)')
ax.set_title('Total Transaction Amount by Month and Cluster')

# Rotate x-axis labels for better readability
plt.xticks(rotation=90)

# Display the plot
plt.show()

# Create a new figure for the number of transactions plot
plt.figure(figsize=(14, 7))

# Create a line plot of the number of transactions per month for each cluster
ax = sns.lineplot(x='Transaction_month_year', y='transaction_count', hue='cluster', data=monthly_transactions)

# Set the x and y-axis labels and plot title
ax.set_xlabel('Month/Year')
ax.set_ylabel('Number of Transactions')
ax.set_title('Number of Transactions by Month and Cluster')

# Rotate x-axis labels for better readability
plt.xticks(rotation=90)

# Display the plot
plt.show()

# In this code, we first convert the TransactionDate column to a datetime object and extract the month and year into a new column called Transaction_month_year. Then, we calculate the total transaction amount and the number of transactions per month for each cluster using groupby() and sum() or size() functions. After that, we create a line plot for both total transaction amount and the number of transactions using sns.lineplot. The x-axis represents the month and year, and the y-axis represents the total transaction amount or the number of transactions. The hue parameter is set to 'cluster', which colors the lines according to the cluster.
plt.figure(figsize=(14, 7))
sns.scatterplot(x='Transaction_month_year', y='Amount_KES', size='days_diff', hue='cluster', data=ctr_1, alpha=0.6, sizes=(20, 200))
plt.xlabel('Transaction Month/Year')
plt.ylabel('Transaction Amount (KES)')
plt.title('Transaction Amounts by Month and Days Difference')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=90)
plt.show()

plt.figure(figsize=(14, 7))
sns.violinplot(x='cluster', y='Amount_KES', data=ctr_1, inner='quartile')
plt.xlabel('Cluster')
plt.ylabel('Transaction Amount (KES)')
plt.title('Distribution of Transaction Amounts by Cluster')
plt.show()
# Calculate the correlation matrix
corr_matrix = ctr_1[['Amount_KES', 'days_diff', 'Transaction_year', 'Transaction_month', 'cluster']].corr()

# Create a clustermap
sns.clustermap(corr_matrix, annot=True, cmap='coolwarm', figsize=(10, 10))
plt.show()

g = sns.PairGrid(ctr_1[['Amount_KES', 'days_diff', 'Transaction_year', 'Transaction_month', 'cluster']], hue='cluster')
g.map_diag(sns.histplot)
g.map_offdiag(sns.scatterplot)
g.add_legend()
plt.show()

sns.pairplot(ctr_1[['Amount_KES', 'days_diff', 'Transaction_year', 'Transaction_month', 'cluster']], hue='cluster')
plt.show()

industry_cluster_count = ctr_1.groupby(['Industry', 'cluster']).size().unstack().fillna(0)

# Normalize the data for better comparison
normalized_industry_cluster_count = industry_cluster_count.div(industry_cluster_count.sum(axis=1), axis=0)

normalized_industry_cluster_count.plot(kind='bar', stacked=True, figsize=(14, 7))
plt.xlabel('Industry')
plt.ylabel('Proportion')
plt.title('Proportion of Clusters by Industry')
plt.legend(title='Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()
from pandas.plotting import parallel_coordinates

# Select the columns you want to plot and normalize the numerical columns
selected_columns = ctr_1[['Amount_KES', 'days_diff', 'Transaction_month', 'cluster']]
normalized_selected_columns = (selected_columns - selected_columns.min()) / (selected_columns.max() - selected_columns.min())

# Keep the 'cluster' column as integers
normalized_selected_columns['cluster'] = selected_columns['cluster']

plt.figure(figsize=(14, 7))
parallel_coordinates(normalized_selected_columns, 'cluster', colormap='viridis')
plt.xlabel('Variables')
plt.ylabel('Normalized Values')
plt.title('Parallel Coordinates Plot of Selected Variables by Cluster')
plt.legend(title='Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()
g = sns.FacetGrid(ctr_1, col='cluster', hue='cluster', height=4, aspect=1)
g.map(sns.scatterplot, 'days_diff', 'Amount_KES', alpha=0.7)
g.add_legend()
plt.subplots_adjust(top=0.8)
g.fig.suptitle('Days Difference vs. Transaction Amount by Cluster')
plt.show()
#### Kmeans Clustering
# import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

# Assume that the ctr_encoded dataframe is already created

# Perform feature scaling (optional but recommended)
scaler = StandardScaler()
ctr_encoded_scaled = scaler.fit_transform(ctr_encoded)

# Determine the optimal number of clusters (k) using the silhouette score
max_clusters = 10  # Adjust this value based on your dataset size and requirements
silhouette_scores = []

for k in range(2, max_clusters + 1):
    kmeans = KMeans(n_clusters=k, random_state=42)
    cluster_labels = kmeans.fit_predict(ctr_encoded_scaled)
    silhouette_avg = silhouette_score(ctr_encoded_scaled, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    print(f"Silhouette score for k={k}: {silhouette_avg}")

optimal_k = silhouette_scores.index(max(silhouette_scores)) + 2
print(f"Optimal number of clusters: {optimal_k}")

# Fit the KMeans model using the optimal number of clusters
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
ctr_encoded['cluster'] = kmeans.fit_predict(ctr_encoded_scaled)

# Check the resulting dataframe with cluster labels
print(ctr_encoded.head())

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Determine the optimal number of clusters (k) using the silhouette score
max_clusters = 10  # Adjust this value based on your dataset size and requirements
silhouette_scores = []

for k in range(2, max_clusters + 1):
    kmeans = KMeans(n_clusters=k, random_state=42)
    cluster_labels = kmeans.fit_predict(ctr_standard_scaled)
    silhouette_avg = silhouette_score(ctr_standard_scaled, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    print(f"Silhouette score for k={k}: {silhouette_avg}")

optimal_k = silhouette_scores.index(max(silhouette_scores)) + 2
print(f"Optimal number of clusters: {optimal_k}")

# Fit the KMeans model using the optimal number of clusters
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
ctr_encoded['cluster'] = kmeans.fit_predict(ctr_standard_scaled)

# Check the resulting dataframe with cluster labels
print(ctr_encoded.head())

from sklearn.decomposition import PCA

# Assume the ctr_encoded dataframe has been clustered using KMeans
# and the resulting cluster labels are stored in the 'cluster' column

# Perform PCA to reduce the dimensions to 2D
pca = PCA(n_components=2)
principal_components = pca.fit_transform(ctr_encoded)

# Create a new dataframe with principal components and cluster labels
pca_df = pd.DataFrame(data=principal_components, columns=['principal_component_1', 'principal_component_2'])
pca_df['cluster'] = ctr_encoded['cluster']

# Plot the clusters
plt.figure(figsize=(10, 6))
for cluster in pca_df['cluster'].unique():
    plt.scatter(pca_df.loc[pca_df['cluster'] == cluster, 'principal_component_1'],
                pca_df.loc[pca_df['cluster'] == cluster, 'principal_component_2'],
                label=f"Cluster {cluster}")

plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend()
plt.title('KMeans Clustering')
plt.show()